<a href="https://colab.research.google.com/github/minah9999/lawbook-in-my-hands-backend/blob/develop/Doc2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 임포트
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd

!pip install konlpy
from konlpy.tag import Komoran

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 58.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# json파일 dataframe으로 읽기
df=pd.read_json('/content/drive/MyDrive/data.json') 
len(df.columns)

In [ ]:
# 행,열 변환
df=df.transpose() 
df.head(3)

In [ ]:
# 지금 행의 index가 "http ..." 을 0,1,2 숫자로 변경
df.reset_index(drop=True, inplace=True)
df.head(3)

In [ ]:
# 15개의 열 중에서 필요없는 열은 제거 (임의로 해놓은 것)
df=df.drop(['http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
         'http://www.aihub.or.kr/kb/law/sentence',
         'http://www.aihub.or.kr/kb/law/courtName',
         'http://www.aihub.or.kr/kb/law/sentenceDate',
         'http://www.aihub.or.kr/kb/law/judgementType',
         'http://www.aihub.or.kr/kb/law/precedentNumber',
         'http://www.aihub.or.kr/kb/law/caseType',
         'http://www.w3.org/2000/01/rdf-schema#label'
         ], axis='columns') 
df.columns

In [ ]:
# 열 이름 알아보기 쉽게 수정
df=df.rename(columns={'http://www.aihub.or.kr/kb/law/caseNumber':'caseNumber',
                      'http://www.aihub.or.kr/kb/law/caseName': 'caseName',
                      'http://www.aihub.or.kr/kb/law/judgementAbstract': 'judgementAbstract',
                      'http://www.aihub.or.kr/kb/law/refArticle': 'refArticle',
                      'http://www.aihub.or.kr/kb/law/precedentText': 'precedentText',
                      'http://www.aihub.or.kr/kb/law/judgementNote': 'judgementNote',
                      'http://www.aihub.or.kr/kb/law/refPrecedent': 'refPrecedent'})
df.columns

In [ ]:
# Nan 값을 ""로 채우기 (참조판례가 Nan인 경우가 있음)
df=df.fillna("")
df.head(3)

In [ ]:
# 현재는 dataframe 값이 list[dict,dict...]
# dict{'type', 'value'}에서 value 값만 뽑아서 dataframe 값에 넣기
# 참고로 list 안에 dict가 여러개 인 경우도 있음
for i in df.index:
 #print(df.loc[i].index)
  for j in range(len(df.loc[i].index)):
    # list안에 dict가 1개일 때
    if len(df.loc[i, df.loc[i].index[j]]) == 1:
      df.loc[i, df.loc[i].index[j]] = df.loc[i, df.loc[i].index[j]][0]['value']
    # list안에 dict가 여러개 일 때
    elif len(df.loc[i, df.loc[i].index[j]]) > 1:
      # value값을 모아서 콤마로 구분하고 string으로 줄거임
      text = ''
      for k in df.loc[i, df.loc[i].index[j]]:
        text += k['value'] + ','
      # 마지막 콤마는 삭제 
      text=text[:-1]
      df.loc[i, df.loc[i].index[j]] = text




In [ ]:
df

텍스트 전처리 과정

In [ ]:
# 토큰화 - 형태소 분석기 사용

# komoran 형태소 분석기 사용
kom = Komoran()

# NNG일반명사, NNP고유명사, VV동사 만 사용
filter_kom = [
              'NNG', #일반명사
              'NNP', #고유명사
              'VV' #동사
]

tokened_judgementNote = []

for judgementNote in df['judgementNote']:
  #형태소 분석만 함
  #judgementNote = kom.morphs(judgementNote) 

  #형태소 분석하고 모든 품사까지 같이 넣음
  #judgementNote = ['/'.join(word) for word in kom.pos(judgementNote)] 

  #형태소 분석하고 filter_kom에 있는 품사에 해당하면 넣음  
  judgementNote = ['/'.join(word) for word in kom.pos(judgementNote) if word[1] in filter_kom] 
  
  tokened_judgementNote.append(judgementNote)

tokened_judgementNote[0]

In [ ]:
# 여러가지 전처리 과정 필요함
# 불용어 제거, 특수문자 제거, 등등등
judgementNote = tokened_judgementNote

In [ ]:
# 리스트에서 각 문장부분 인덱싱?????
index_judgementNote = []
for i in range(len(df)):
  index_judgementNote.append([judgementNote[i],i])

In [ ]:
index_judgementNote[:3]

In [ ]:
# DOc2Vec에서 사용하는 태그문서형으로 변경
tagged_judgementNote = [TaggedDocument(d, [int(c)]) for d,c in index_judgementNote]

In [ ]:
tagged_judgementNote[:3]

모델 생성

In [ ]:
    # make model
import multiprocessing
cores = multiprocessing.cpu_count()
d2v_faqs = doc2vec.Doc2Vec(
                                vector_size=200, #벡터 차원의 크기
#                                 alpha=0.025,
#                                 min_alpha=0.025,
                                hs=1,
                                negative=0,
                                dm=0, #학습 방법 1=PV-DM, 0=PV-DBOW
                                dbow_words = 1,
                                min_count = 5, #학습에 사용할 최소 단어 빈도 수
                                workers = cores,
                                seed=0,
                                epochs=20
                                )

d2v_faqs.build_vocab(tagged_judgementNote)

d2v_faqs.train(tagged_judgementNote,
               total_examples = d2v_faqs.corpus_count,
               epochs = d2v_faqs.epochs
                                )

In [ ]:
# 테스트하는 문장도 같은 전처리를 해준다.
test = '방송사가 술에 취해 길에 쓰러졌던 40대 남자가 정신병원에 4년간이나 강제수용된 사실을 보도하면서, 정신보건법령상 제도의 운영상 문제점을 부각시키기 위하여 사실관계를 단순화시켜 그 일부 측면만을 강조하는 과정에서 취재된 정신병원의 사정을 방송 내용에 포함시키지 않았다 하더라도, 전체적인 맥락에서 방송 내용의 중요 부분이 진실에 합치함을 이유로 정정보도청구의 요건을 갖추지 못하였다고 한 사례'
tokened_test = ['/'.join(word) for word in kom.pos(test)]

test = tokened_test

In [ ]:
topn = 5
test_vector = d2v_faqs.infer_vector(test)
result_list = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)

for i in range(topn):
  print("{}위. {}, {} {}".format(i+1, result_list[i][1], result_list[i][0],df['judgementNote'][result_list[i][0]] ))

In [ ]:
import os

In [ ]:
d2v_faqs.save(os.path.join('data','/content/drive/MyDrive/d2v_judge_size200_min5_epoch20.model'))